## !! Все примеры нужно запускать не отсюда, а из скриптов

**Multiprocessing** позволяет создавать процессы, которые выполняются полностью независимо друг от друга

Способы создания нового процесса в multiprocessing:

- fork
- spawn
- forkserver

## Контекст (Context)

In [ ]:
import multiprocessing as mp

if __name__ == '__main__':
    context_spawn = mp.get_context('spawn')
    context_fork = mp.get_context('fork')

## Процесс (Process)

In [ ]:
from multiprocessing import Process

def print_one_name(name):
    print('Name:', name)

if __name__ == '__main__':
    p = Process(target=print_one_name, args=('Bob',))
    p.start()
    p.join()

## Создание процесса через fork

- Инициализируется объект класса Popen [при старте]
- os.fork
- Вызов пользовательской функции

## Создание процесса через spawn

- Инициализируется объект класса Popen [при старте]
- Собирается информация о родительском процессе
- Сериализуется словарь с информацией о родительском процессе и сам объект процесса
- Создается команда для запуска интепретатора, в которую передаются все нужные аргументы
- В дочерний процесс передаётся сериализованная информация о процессе


## Завершение процесса (join)

- родительский процесс дожидается завершения дочернего
- происходит несколько проверок
- вызывается метод wait

## Пул процессов

- Pool.apply() - вызывает функцию с аргументами

- Pool.apply_async() - асинхронный вариант Pool.apply(). То есть apply_async() не дожидается результата завершения работы функции

- Pool.map() - многопроцессорный аналог встроенной функции map(), которая применяет функцию к любой последовательности, поддерживающей итерирование, и возвращает список результатов работы этой функции.

- Pool.map_async() - асинхронный вариант map()

In [ ]:
import time  
from multiprocessing.pool import Pool  
  
def wait_a_second(x):  
    time.sleep(1)  
    return x  
  
if __name__ == "__main__":  
    with Pool(4) as pool:  
        result = pool.map(wait_a_second, [1,2,3,4])  
        print(result)

## Передача данных при создании процесса

In [ ]:
import multiprocessing as mp

def test(targetList, newElem):
    targetList.append(newElem)
    print(new_Elem, 'added')


if __name__ == '__main__':
    myList = [1, 0]
    test(myList, 4)
    
    print('List after just func', myList)

    p = mp.Process(target=test, args=[myList, 5]) 
   

    p.start()
    p.join()
    print('List after multiproc func', myList)


## Shared Memory

In [ ]:
import multiprocessing as mp

def sharedMem(targetNum, targetArr):
    targetNum.value = 228
    for i in range(len(targetArr)):
        targetArr[i] = -targetArr[i]

if __name__ == '__main__':
    myNum = mp.Value('d', 0.0)
    myArr = mp.Array('i', range(10))

    p = mp.Process(target=sharedMem, args=(myNum, myArr))
    p.start()
    p.join()

    print(myNum.value)
    print(myArr[:])

## Server process. SyncManager

In [ ]:
import multiprocessing as mp

def test(targetDict, targetList):
    targetDict[1] = '2'
    targetDict['3'] = 2
    targetDict[0.25] = 0
    targetList.reverse()

if __name__ == '__main__':
    with mp.Manager() as manager:
        myDict = manager.dict()
        myList = manager.list(range(10))

        p = mp.Process(target=test, args=(myDict, myList))
        p.start()
        p.join()

        print(myDict)
        print(myList)

## Proxy-объекты

In [ ]:
manager.list([1,2,3]) == [1,2,3] -> False
manager.list([1,2,3])[:] == [1,2,3] -> True

## Имплементация пользовательского прокси и менеджера

In [ ]:
from multiprocessing.managers import BaseManager

class MyManager(BaseManager):
    pass

In [ ]:
class MyClass:

    def __init__(self):
        self.x = 10

    def __str__(self):
        return str(self.x)
    
    def change(self):
        self.x += 200

In [ ]:
from multiprocessing.managers import NamespaceProxy

class MyClassProxy(NamespaceProxy):
    pass

In [ ]:
MyManager.register('MyClassRegistered', MyClass, MyClassProxy)

M = MyManager()
M.start()
MC = M.MyClassRegistered()

In [ ]:
def changer(obj):
    obj.change()

if __name__ == '__main__':
    M = MyManager()
    M.start()

    MC = M.MyClassRegistred()
    p = mp.Process(target = changer, args=(MC,))
    p.start()
    p.join()
    print('Here I am: ', MC.x)